<img src="https://github.com/StatQuest/signa/blob/main/chapter_03/images/final_nn.png?raw=1" alt="a neural network with multiple inputs and outputs" style="width: 600px;">

In [2]:
%%capture
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## NOTE: If you **don't** need to install anything, you can comment out the
##       next line.
##
##       If you **do** need to install something, just know that you may need to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu
##
##       Also, installing can take a few minutes, so go get yourself a snack!
!pip install lightning

In [43]:
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

import pandas as pd # We'll use pandas to read in the data and normalize it
from sklearn.model_selection import train_test_split # We'll use this to create training and testing datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris

The Iris flower dataset is a classic dataset originally made famous by Rondal Fisher in 1936, and has since been used countless times to demonstrate the effectiveness of various classification algorithms. The dataset consists of 150 samples total, 50 for each of 3 species of Iris, Setosa, Versicolor, and Virginica. Each row in the dataset contains measurements for 4 variables: petal width and length and sepal width and length.

**Read in the dataset with the pandas function read_table()**

In [51]:
#url = "https://raw.githubusercontent.com/StatQuest/signa/main/chapter_03/iris.txt"
#df = pd.read_table(url, sep=",", header=None)
iris= load_iris(as_frame=True)
df=iris.data


In [52]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [59]:
df.shape

(150, 4)

Now, let's split the data into training and testing datasets. The first step is to separate the columns into input values and labels.
One DataFrame will have the petal and sepal widths, the values we will use to make predictions, and we'll call this DataFrame input_values. The other DataFrame will have the species, the values we will use to determine how good those predictions are, and this DataFrame will be called label_values.

In [56]:
input_values = df[['petal width (cm)', 'sepal width (cm)']]
input_values.head()

,petal width (cm),sepal width (cm)
0,0.2,3.5
1,0.2,3.0
2,0.2,3.2
3,0.2,3.1
4,0.2,3.6


In [57]:
label_values = iris.target
label_values.head()

,target
0,0
1,0
2,0
3,0
4,0


In [58]:
## To determine the number of iris species in the dataset,
## we'll count the number of unique values in the column called `class`.
label_values.nunique()

3

## **factorize()**
It returns two things:
*   A list of numeric codes (same shape as your input).
*   An array of unique values (so you know what each number represents).






In [12]:
classes_as_numbers, classes = label_values.factorize()

In [13]:
classes_as_numbers ## print out the numbers

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
input_train, input_test, label_train, label_test = train_test_split(input_values,
                                                                    classes_as_numbers,
                                                                    test_size=0.25,
                                                                    stratify=classes_as_numbers)

In [15]:
input_train.shape

(112, 2)

In [16]:
input_test.shape

(38, 2)

Now, because our neural network will have 3 outputs, one for each species (see the drawing of the neural network above), we need to convert the numbers in label_train into 3 element arrays, where each element in an array corresponds to a specific output in the neural network. Specifically, we'll use [1.0, 0.0, 0.0] to correspond to Setosa, [0.0, 1.0, 0.0] for Versicolor, and [0.0, 0.0, 1.0] for Virginica. The good news is that we can easily do the one-hot encoding. We also tack on type(torch.float32) to ensure the numbers are saved in the correct format for the neural network to process efficiently.

In [17]:
one_hot_label_train = F.one_hot(torch.tensor(label_train)).type(torch.float32)

In [18]:
one_hot_label_train[:10]

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

Now, let's normalize the input variables so that their values range from 0 to 1. Normalizing data, so that it's all on the same scale, often makes it easier to train machine learning methods. In this case, since we have two datasets, input_train and input_test, we'll start determining the maximum and minimum values in input_train. Then we will use those values to normalize input_train and input_test. Using the maximum and minimum values from input_train to normalize both datasets avoids something called Data Leakage.

In [19]:
## First, determine the maximum values in input_train...
max_vals_in_input_train = input_train.max()
## Now print them out...
max_vals_in_input_train

,0
petal_width,2.5
sepal_width,4.4


In [20]:
# Initialize the scaler
scaler = MinMaxScaler()

In [21]:
input_train_normalized = scaler.fit_transform(input_train)


In [22]:
input_test_normalized = scaler.fit_transform(input_test)

let's put our training data into a DataLoader, which we can use to train the neural network. DataLoaders are great for large datasets because they make it easy to access the data in batches, make it easy to shuffle the data each epoch, and they make it easy to use a relatively small fraction of the data if we want to do a quick and dirty training for debugging our code.

To put our data training data into a DataLoader, we'll start by converting input_train into tensors with torch.tensor(). We'll then combine 'input_train with one_hot_label_train to create a TensorDataset. Lastly, we'll use the TensorDataset to create the DataLoader.

In [23]:
## Convert the DataFrame input_train into tensors
input_train_tensors = torch.tensor(input_train.values).type(torch.float32)

## now print out the first 5 rows to make sure they are what we expect.
input_train_tensors[:5]

tensor([[0.2000, 3.4000],
        [0.1000, 3.0000],
        [1.5000, 3.0000],
        [0.2000, 3.8000],
        [2.1000, 3.0000]])

In [24]:
# Convert the DataFrame input_test into tensors
input_test_tensors = torch.tensor(input_test.values).type(torch.float32)

## now print out the first 5 rows to make sure they are what we expect.
input_test_tensors[:5]

tensor([[1.0000, 2.2000],
        [1.8000, 3.0000],
        [0.2000, 3.2000],
        [2.3000, 3.2000],
        [0.1000, 4.1000]])

In [25]:
train_dataset = TensorDataset(input_train_tensors, one_hot_label_train)
train_dataloader = DataLoader(train_dataset)

<a id="build"></a>
# Building a neural network with multiple inputs and outputs with PyTorch and Lightning

Building a neural network with PyTorch means creating a new class. And to make it easy to train the neural network, this class will inherit from `LightningModule`.

Our new class will have the following methods:
- `__init__()` to initialize the Weights and Biases and keep track of a few other housekeeping things.
- `forward()` to make a forward pass through the neural network.
- `configure_optimizers()` to configure the optimizer. There are lots of optimizers to choose from, but in this tutorial, we'll change things up and use `Adam`.
- `training_step()` to pass the training data to `forward()`, calculate the loss and keep track of the loss values in a log file.

Also, for reference, here is a picture of the neural network we want to create:
<img src="https://github.com/StatQuest/signa/blob/main/chapter_03/images/final_nn.png?raw=1" alt="a neural network with multiple inputs and outputs" style="width: 800px;">

In [26]:
class MultipleInsOuts(L.LightningModule):

  def __init__(self):
    super().__init__()

    L.seed_everything(seed=42)
    self.input_to_hidden=nn.Linear(in_features=2,out_features=2,bias=True)
    self.hidden_to_output = nn.Linear(in_features=2, out_features=3, bias=True)
    self.loss = nn.MSELoss(reduction='sum')

  def forward(self,input):

    ## First, we run the input values to the activation functions
        ## in the hidden layer
        hidden = self.input_to_hidden(input)
        ## Then we run the values through a ReLU activation function
        ## and then run those values to the output.
        output_values = self.hidden_to_output(torch.relu(hidden))
        return(output_values)

  def configure_optimizers(self):
        ## In this example, configuring the optimizer
        ## consists of passing it the weights and biases we want
        ## to optimize, which are all in self.parameters(),
        ## and setting the learning rate with lr=0.001.
        return Adam(self.parameters(), lr=0.001)

  def training_step(self, batch, batch_idx):
        ## The first thing we do is split 'batch'
        ## into the input and label values.
        inputs, labels = batch

        ## Then we run the input through the neural network
        outputs = self.forward(inputs)

        ## Then we calculate the loss.
        loss = self.loss(outputs, labels)

        ## Lastly, we could add the loss a log file
        ## so that we can graph it later. This would
        ## help us decide if we have done enough training
        ## Ideally, if we do enough training, the loss
        ## should be small and not getting any smaller.
        # self.log("loss", loss)

        return loss

In [27]:
model = MultipleInsOuts() # First, make model from the class

## Now print out the name and value for each named parameter
## parameter in the model. Remember parameters are variables,
## like Weights and Biases, that we can train.
for name, param in model.named_parameters():
    print(name, torch.round(param.data, decimals=2))

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


input_to_hidden.weight tensor([[ 0.5400,  0.5900],
        [-0.1700,  0.6500]])
input_to_hidden.bias tensor([-0.1500,  0.1400])
hidden_to_output.weight tensor([[-0.3400,  0.4200],
        [ 0.6200, -0.5200],
        [ 0.6100,  0.1300]])
hidden_to_output.bias tensor([0.5200, 0.1000, 0.3400])


Training our new neural network means we create a model from the new class, `MultipleInsOuts`...

In [28]:
model = MultipleInsOuts()

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


...and then create a **Lightning Trainer**, `L.Trainer`, and use it to optimize the parameters. **NOTE:** We will start with 10 epochs, complete runs through our training data. This may be enough to successfully optimize all of the parameters, but it might not. We'll find out later in the tutorial when we make a graph of how the loss values change during training.

In [60]:
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, train_dataloaders=train_dataloader)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name             | Type    | Params | Mode 
-----------------------------------------------------
0 | input_to_hidden  | Linear  | 6      | train
1 | hidden_to_o

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [30]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)

In [34]:
predictions[0:4,]

tensor([[ 0.3530,  0.2707,  0.2541],
        [ 0.1200,  0.5360,  0.4787],
        [ 0.7014, -0.0538,  0.1258],
        [-0.0448,  0.7117,  0.6030]], grad_fn=<SliceBackward0>)

In [35]:
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies argmax to rows, dim=1 applies argmax to columns
predicted_labels[0:10] # print out the first 4 predictions

tensor([0, 1, 0, 1, 0, 0, 0, 0, 1, 2])

And we see that our neural network only correctly predicts 74% of the testing data. This isn't very good. So, will training our model for more epochs improve the model's predictions?

One way to answer that question is to just train for longer and see what happens.

The good news is that because we're using Lightning, we can pick up where we left off training without starting over from scratch. This is because training with Lightning creates checkpoint files that keep track of the Weights and Biases as they change. As a result, all we have to do to pick up where we left off is tell the Trainer where the checkpoint files are. This is awesome and will save us a lot of time since we don't have to retrain the first 10 epochs. So, let's add an additional 90 epochs to the training.

To add additional epochs to the training, we first identify where the checkpoint file is with the following command.

In [36]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path ## By default, "best" = "most recent"

Then we create a new Lightning Trainer, just like before, but we set the number of epochs to 100. Given that we already trained for 10 epochs, this means we'll do 90 more.

In [37]:
## First, create a new Lightning Trainer
trainer = L.Trainer(max_epochs=100) # Before, max_epochs=10, so, by setting it to 100, we're adding 90 more.

## Then call trainer.fit() using the path to the most recent checkpoint files
## so that we can pick up where we left off.
trainer.fit(model, train_dataloaders=train_dataloader, ckpt_path=path_to_checkpoint)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Restoring states from the checkpoint path at /content/lightning_logs/version_0/checkpoints/epoch=9-step=1120.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [61]:
# Run the input_test_tensors through the neural network
predictions = model(input_test_tensors)

## Select the output with highest value...
predicted_labels = torch.argmax(predictions, dim=1) ## dim=0 applies softmax to rows, dim=1 applies softmax to columns

## Now compare predicted_labels with test_labels to calculate accuracy
## NOTE: torch.eq() computes element-wise equality between two tensors.
##       label_test, however, is just an array, so we convert it to a tensor
##       before passing it in. torch.sum() then adds up all of the "True"
##       output values to get the number of correct predictions.
##       We then divide the number of correct predictions by the number of predicted values,
##       obtained with len(predicted_labels), to get the percentage of correct predictions
torch.sum(torch.eq(torch.tensor(label_test), predicted_labels)) / len(predicted_labels)

tensor(0.8947)

Now that our model is trained, we can use it to make predictions from new data. This is done by passing the model a tensor with normalized petal and sepal widths wrapped up in a tensor.

For example, if the raw petal and sepal width measurements were 0.2 and 3.0, we would first normalize them using the maximum and minimum values we calculated with the training data.